# Computer Vision with Model Caching

<table align="left">
    <td>
        <a href="https://colab.research.google.com/github/dreoporto/ptmlib/blob/main/ptmlib/notebooks/Computer-Vision-with-Model-Caching.ipynb" target="_blank">
            <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
        </a>
    </td>
</table>    

This notebook demonstrates using the PTMLib `model_tools.load_or_fit_model()` function to train and save a Computer Vision model.

You can learn more about PTMLib (Pendragon Tools for Machine Learning) at https://github.com/dreoporto/ptmlib

This example was originally derived from the "DeepLearning.AI TensorFlow Developer" course notebook at https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%204%20-%20Lesson%202%20-%20Notebook.ipynb

It has been enhanced to include:

- examples of PTMLib usage
- a dropout layer
- `validation_split` usage
- early stopping using a callback function

## Setup

### Import common libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Import PTMLib

In [ ]:
# UNCOMMENT pip install BELOW IF PTMLib IS NOT INSTALLED
!pip install --no-index -f https://github.com/dreoporto/ptmlib/releases ptmlib

from ptmlib.time import Stopwatch, AlertSounds
import ptmlib.model_tools as modt

### Check Setup

In [ ]:
print('TF VERSION:', tf.__version__)

## Setup PTMLib Timers

Use `Stopwatch` to alert you when work completes, including the entire notebook.

In [ ]:
main_stopwatch = Stopwatch()
main_stopwatch.start()

## Get MNIST Data

In [ ]:
mnist = keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

### Check some sample data to verify

In [ ]:
np.set_printoptions(linewidth=200)
plt.imshow(training_images[0])
plt.show()

In [ ]:
print(training_labels[0])

### Normalize the image data

Normalize image data to values between 0 and 1

In [ ]:
training_images = training_images / 255.0
test_images = test_images / 255.0

## Create the Keras model

### Setup Hyperparameters 1

In [ ]:
hp_dropout = 0.2

### Create a Sequential model using Keras

In [ ]:
# I <3 Keras

model = keras.models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dropout(hp_dropout),
    layers.Dense(512, activation=tf.nn.relu),
    layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

## Create an Early Callback function

Avoid excessive training by setting an accuracy target

In [ ]:
class MyCallback(keras.callbacks.Callback):

    def __init__(self, target):
        super().__init__()
        self.target = target

    def on_epoch_end(self, _, logs=None):

        if logs is None:
            logs = {}
        if logs.get("accuracy") > self.target:
            print(f"\nReached {self.target * 100}% accuracy so cancelling training!")
            self.model.stop_training = True

## Train your model, with caching

### Setup Hyperparameters 2

In [ ]:
hp_epochs = 50
hp_target = 0.91
hp_validation_split = 0.2

### Create a `fit` lambda function

Create a lambda function that will be called to fit your model

In [ ]:
fit_model_function_with_callback = lambda my_model, x, y, validation_data, epochs: my_model.fit(
        x, y, validation_data, epochs=epochs, callbacks=[early_callback], validation_split=hp_validation_split)

### Train your model

If you run this notebook a second time, it will load the model and charts from the saved files.  No need to worry about shutting down Jupyter, or rebooting your machine (i.e. patches) !

In [ ]:
model_file_name = "computer_vision_1"
early_callback = MyCallback(target=hp_target)

model, history = modt.load_or_fit_model(model, model_file_name, x=training_images, y=training_labels, 
                                        epochs=hp_epochs, fit_model_function=fit_model_function_with_callback,
                                        metrics=["accuracy"])

## Evaluate your model

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
classifications = model.predict(test_images)

In [ ]:
print(classifications[0])
print(test_labels[0])
print(max(classifications[0]))

## Using the Cached Model and History

We will now run this function again to show what happens if you restart this notebook (ex: using *Kernel > Restart & Run All* in Jupyter)

In [ ]:
model, history = modt.load_or_fit_model(model, model_file_name, x=training_images, y=training_labels, 
                                        epochs=hp_epochs, fit_model_function=fit_model_function_with_callback,
                                        metrics=["accuracy"])

Since history data is also cached, we can create additional plots for further analysis.

In [ ]:
# let's combine both charts

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Accuracy/Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend(["train_acc", "val_acc", "loss", "val_loss"])
plt.grid()
plt.show()

## Removing Cached Files

To run `load_or_fit_model` from scratch, without using the saved model and images, simply delete the following files, each of which contains *computer_vision_1* in the file name.  This was set using the `model_file_name` variable.

- computer_vision_1.h5
- accuracy-computer_vision_1.png
- loss-computer_vision_1.png
- computer_vision_1_history.pkl

This is especially important if you are still adjusting your model layers or hyperparameters for model optimization.

You can also change the value of the `model_file_name` variable if you wish to save the results from multiple runs.  This is useful when comparing performance graphs.

In [ ]:
# ALL DONE!

main_stopwatch.stop(sound_path=AlertSounds.DORE)